<a href="https://colab.research.google.com/github/lainey-reed/Textbook/blob/master/Colab/AC_Textbook_Chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 6: Flow Control and Measumrent

All of the examples in this document are from [Chapter 6](https://aguaclara.github.io/Textbook/Flow_Control_and_Measurement/FCM_Intro.html#flow-control-and-measurement-introduction)

In [ ]:
!pip install aguaclara

## Example 1: Kinematic Viscosity of Coagulants

The following example is from the chapter section [Flow Control and Measurement Design](https://aguaclara.github.io/Textbook/Flow_Control_and_Measurement/FCM_Design.html#kinematic-viscosity-of-coagulants)


In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import numpy as np
import matplotlib.pyplot as plt
# the following concentrations were prepared for each chemical
Measured_C = np.linspace(0,600,13) * u.g/u.L
Measured_C[0] = 10 * u.g/u.L
# the following dynamic viscosities were measured
Measured_Viscosity_Dynamic_Alum = np.array([0.00097, 0.00114,0.00143,0.00178,0.00208,0.00253,0.00349,0.00451,0.00532,0.00691,0.00901,0.0117,0.0133]) * u.kg/u.s/u.m
Measured_Viscosity_Dynamic_PACl = np.array([0.001,0.00108,0.00122,0.00145,0.00171,0.00231,0.00256,0.00301,0.00392,0.00378,0.00476,0.00486,0.00768]) * u.kg/u.s/u.m
Density_PACl_400gperL = 58.88 * u.g/(50 * u.mL)
Density_Alum_600gperL = 63.2 * u.g/(50 * u.mL)
temperature = 22 * u.degC

def Density_Alum(Alum_C, temperature):
  #Alum_C is concentration of aluminum sulfate with attached waters (not concentration of aluminum)
  return Density_Alum_600gperL * Alum_C/(600 * u.g/u.L) + pc.density_water(temperature)*(600 * u.g/u.L - Alum_C)/(600 * u.g/u.L)

def Density_PACl(PACl_C, temperature):
  #PACl_C is concentration of poly aluminum chloride granules (not concentration of aluminum
  return Density_PACl_400gperL * PACl_C/(400 * u.g/u.L) + pc.density_water(temperature)*(400 * u.g/u.L - PACl_C)/(400 * u.g/u.L)

Measured_Viscosity_Kinematic_Alum = Measured_Viscosity_Dynamic_Alum/Density_Alum(Measured_C, temperature)
Measured_Viscosity_Kinematic_PACl = Measured_Viscosity_Dynamic_PACl/Density_PACl(Measured_C, temperature)
def Alum_Viscosity_Kinematic(Alum_C, temperature):
  return (1 + (4.225 * 10**(-6) * ((Alum_C.to(u.kg/u.m**3)).magnitude)**2.289))*pc.viscosity_kinematic_water(temperature)

def PACl_Viscosity_Kinematic(PACl_C, temperature):
  return (1 + (2.383 * 10**(-5) * ((PACl_C.to(u.kg/u.m**3)).magnitude)**1.893))*pc.viscosity_kinematic_water(temperature)

fig, ax = plt.subplots()
ax.plot(Measured_C, Measured_Viscosity_Kinematic_Alum.to(u.mm**2/u.s), 'ro')
ax.plot(Measured_C, Alum_Viscosity_Kinematic(Measured_C, temperature).to(u.mm**2/u.s), 'r-')
ax.plot(Measured_C, Measured_Viscosity_Kinematic_PACl.to(u.mm**2/u.s), 'bo')
ax.plot(Measured_C, PACl_Viscosity_Kinematic(Measured_C, temperature).to(u.mm**2/u.s), 'b-')

ax.set(xlabel=r'Coagulant concentration prepared from granules ' + r'$\left[\frac{g}{L}\right]$')
ax.set(ylabel='Kinematic Viscosity ' + r'$\left[\frac{mm^2}{s}\right]$')
ax.legend(['Alum data','Alum model', 'PACl data','PACl model'])
#fig.savefig('Flow_Control_and_Measurement/Images/Coagulant_Viscosity')
plt.show()

## Example 2: Design Challenge

The following code segments are from the design challenge in the section [Flow Control and Measurement Solution](https://aguaclara.github.io/Textbook/Flow_Control_and_Measurement/FCM_Solution.html)

In [ ]:
import aguaclara as ac
from aguaclara.core.units import unit_registry as u
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print('The *vena contracta* coefficient for an orifice is ',ac.VC_ORIFICE_RATIO,'.')

In [ ]:
WaterElevationNormalized = np.linspace(-1,2,100)
DiamOrifice = 5*u.cm
WaterElevation = WaterElevationNormalized*DiamOrifice

#Here we initialize some empty numpy arrays of the same size and type as WaterElevationNormalized
HorizontalOrificeFlows = np.empty_like(WaterElevationNormalized)
VerticalOrificeFlows = np.empty_like(WaterElevationNormalized)

#Here we need to populate the arrays we created above.
# Our graphing library, pyplot, cannot handle units, so we need to remove
# them from each element as we insert it into the array.
# While the code looks somewhat ugly, this is the best place to remove units;
# we are removing them after all calculations have been made to minimize the
# severity of floating-point errors.
for i in range(len(WaterElevation)):
  HorizontalOrificeFlows[i] = (ac.flow_orifice(DiamOrifice, WaterElevation[i], ac.VC_ORIFICE_RATIO).to(u.L/u.s).magnitude)
  VerticalOrificeFlows[i] = (ac.flow_orifice_vert(DiamOrifice, WaterElevation[i], ac.VC_ORIFICE_RATIO).to(u.L/u.s).magnitude)

fig, ax = plt.subplots()
ax.plot(WaterElevationNormalized, HorizontalOrificeFlows, 'r-', WaterElevationNormalized, VerticalOrificeFlows, 'b-')

ax.set(xlabel='Normalized height of water above center of the orifice')
ax.set(ylabel='Flow rate through the orifice (L/s)')
ax.legend(['Horizontal Orientation', 'Vertical orientation'])
ax.grid(True)
#fig.savefig('Flow_Control_and_Measurement/Images/Horizontal_vs_Vertical_Orifice_Orientation')
plt.show()

In [ ]:
FlowPlant = 50*u.L/u.s
T = 20 * u.degC
NuBleach = ac.viscosity_kinematic_water(T)
HeadlossDosingTubeMax = 20*(u.cm)
StockCl2 = 51.4*(u.g/u.L)
DoseCl2 = 2*(u.mg/u.L)
RatioError = 0.1
KMinor = 2

In [ ]:
FlowStockClMax = (FlowPlant * DoseCl2 / StockCl2).to(u.mL/u.s)
print('The required flow of bleach is', FlowStockClMax)

In [ ]:
print('The daily required flow of bleach is',FlowStockClMax.to(u.L/u.day))

In [ ]:
DiamTubeArray = (np.arange(1,6,1)) * 1 / 16 * u.inch
print(DiamTubeArray.magnitude, DiamTubeArray.units)

In [ ]:
VelTubeMax = (((RatioError * 2 * HeadlossDosingTubeMax * u.gravity) / KMinor)**(1/2)).to(u.m/u.s)
print('The maximum average velocity in a dosing tube is', VelTubeMax)

In [ ]:
HeadlossMinorMax = RatioError * HeadlossDosingTubeMax
print('The head loss due to minor losses when the tube is at maximum capacity is', HeadlossMinorMax)

In [ ]:
def flow_cdc_max(diam, VelTubeMax):
    Flow = ac.area_circle(diam) * (VelTubeMax)
    return Flow

FlowMaxArray = flow_cdc_max(DiamTubeArray, VelTubeMax).to(u.mL/u.s)
print(FlowMaxArray)

In [ ]:
NDosingTubes = np.ceil(FlowStockClMax / FlowMaxArray)
print('The number of tubes of each diameter is', NDosingTubes)

In [ ]:
FlowDosingTubeArray = FlowStockClMax / NDosingTubes
print('The flow rate per tube is', FlowDosingTubeArray)
print(FlowStockClMax)

In [ ]:
def length_tube(flow_max, diam, headloss_max, nu, k_minor):
    "Returns the length of tube necessary to handle the maximum flow."
    L = (((ac.core.constants.GRAVITY * headloss_max * np.pi * diam**4)/
                           (128 * nu * flow_max))-
                    ((k_minor * flow_max)/
                             (16 * np.pi * nu)))
    return L

LengthDosingTube = length_tube(FlowDosingTubeArray, DiamTubeArray,
                                 HeadlossDosingTubeMax, NuBleach, KMinor).to(u.m)

print('The length of each dosing tube would be', LengthDosingTube)

In [ ]:
MYPICK = 1
print('The number of dosing tubes I will need is',NDosingTubes[MYPICK])
print('The flow through each tube is', FlowDosingTubeArray[MYPICK])
print('The inner diameter of the tube is', DiamTubeArray[MYPICK])
print('The length of each tube is', LengthDosingTube[MYPICK].to(u.m))
